# Patient Data Query

This notebook provides functions to query patient data from multiple databases for a given prontuario.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_embrioes_congelados
- **Finops Silver Layer** (huntington_data_lake.duckdb):
  - silver.mesclada_vendas

## Usage:
```python
# Query data for a specific prontuario
results = query_patient_data(prontuario='160751')
```


In [51]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Database Connection Functions


In [52]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Individual Table Query Functions


In [53]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
        if not df.empty:
            display(df)
        else:
            print("No records found.")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [54]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_mesclada_vendas(conn, prontuario):
    """Query silver.mesclada_vendas for a specific prontuario"""
    print(f"\n=== SILVER.MESCLADA_VENDAS ===")
    query = f"""
    SELECT * FROM silver.mesclada_vendas 
    WHERE prontuario = {prontuario}
    ORDER BY "DT Emissao" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.mesclada_vendas")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [55]:
def query_patient_data(prontuario):
    """
    Query all patient data from multiple databases for a given prontuario.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results
    """
    print(f"\n{'='*80}")
    print(f"QUERYING DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_embrioes = query_embrioes_congelados(clinisys_con, prontuario)
        
        # Query finops table
        df_mesclada = query_mesclada_vendas(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes)} records")
        print(f"- silver.mesclada_vendas: {len(df_mesclada)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'embrioes_congelados': df_embrioes,
            'mesclada_vendas': df_mesclada
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


In [56]:
# Example usage: Query data for a specific prontuario
prontuario = '160751'  # Change this to the desired prontuario

# Query all patient data
results = query_patient_data(prontuario)

# Access individual DataFrames
print(f"\nAccessing individual results:")
print(f"Micromanipulacao records: {len(results['micromanipulacao'])}")
print(f"Micromanipulacao oocitos records: {len(results['micromanipulacao_oocitos'])}")
print(f"Tratamentos records: {len(results['tratamentos'])}")
print(f"Embrioes congelados records: {len(results['embrioes_congelados'])}")
print(f"Mesclada vendas records: {len(results['mesclada_vendas'])}")



QUERYING DATA FOR PRONTUARIO: 160751
Query executed at: 2025-10-02 13:10:33.629892

=== VIEW_MICROMANIPULACAO ===
Found 2 records in view_micromanipulacao


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,26549,2451/25,160751,42,39,2025-07-23,None,13:05,13:15,10:20,10,3492,None,Não,None,None,None,12:10,08:00,None,08:00,08:00,08:00,None,6,9,None,None,None,None,None,Não,None,1,None,None,2025-07-23,ICSI,06 espermatozoides móveis progressivos\r\n,06 espermatozoides normais\r\n,None,None,None,None,None,None,0000000001,<NA>,<NA>,<NA>,3755,3692,None,3492,


=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 15 records in view_micromanipulacao_oocitos


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,105080,12102,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,edc82079d77a7df6b2366c435da905d4,2025-07-21 21:40:02,0,<NA>
1,105081,12102,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7f4fe67dc5667b3b95859bc0778d97ae,2025-07-21 21:40:02,0,<NA>
2,105082,12102,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,759502c2f2b7d06b6271950ba22d7cb0,2025-07-21 21:40:02,0,<NA>
3,105083,12102,Não,GV,None,None,None,None,Descartado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,eb7fcedcb948086b11b0fbd839315d8d,2025-07-21 21:40:02,0,<NA>
4,105084,12102,Não,GV,None,None,None,None,Descartado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,415a63fbb69b83a7906b9f60133bef59,2025-07-21 21:40:02,0,<NA>
5,105085,12102,Não,GV,None,None,None,None,Descartado,None,None,None,None,None,C0,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0bb32c8feb217cb3080bcb41d01e099c,2025-07-21 21:40:02,0,<NA>
6,265872,26549,Não,MII,None,None,3548,None,Descartado,None,None,None,None,1CP,None,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,97371e4d615aca722ab027fa28adc566,2025-07-28 19:05:31,1,1
7,265873,26549,Não,MII,None,None,3548,None,Descartado,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,6,2,Simétricos,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,a322eaa21774429503e362e662e1026d,2025-07-28 19:05:31,1,2
8,265874,26549,Não,MII,None,None,3548,None,Descartado,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3c746a6ebccc206306a1a5f31d11cf70,2025-07-28 19:05:31,1,3
9,265875,26549,Não,MII,None,None,3548,3492,Criopreservado,Sim,Sim,None,PGT-A,2PN,C0,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,7,1,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 1,None,None,Sim,None,Blastocisto Grau 3,C,C,None,None,None,None,None,N.I,Sim,8d8d5c27b43172c392415fe20a3dff1c,2025-08-18 19:04:55,1,4



=== VIEW_TRATAMENTOS ===
Found 2 records in view_tratamentos


...


=== VIEW_EMBRIOES_CONGELADOS ===
Found 1 records in view_embrioes_congelados


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,cores,embriao,doado,lote,celulas,qualidade,qualidade_recongelamento,comentarios,comentarios_descongelamento,ah,dia2,maturidade,dia2_2,dia2_3,dia3,dia3_2,dia3_3,dia4,dia4_2,dia4_3,dia4_4,gd,gd2,gd3,dia5,dia5_2,dia5_3,dia5_4,dia6,dia6_2,dia6_3,dia6_4,dia7,dia7_2,dia7_3,dia7_4,clivou_d2,clivou_d3,clivou_d4,clivou_d5,clivou_d6,clivou_d7,transferidos,pgd,resultado_pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,observacao,destino,hash,extraction_timestamp
0,88094,265875,29636,0,160751,P4,E23018,<NA>,#4,None,None,None,Blastocisto Grau 3 - C - C,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,None,2.1,D6,N.I,AN,5,E23018,None,<NA>,c0a5486c5a7ca26a225190f413fcb429,2025-08-18 19:02:54



=== SILVER.MESCLADA_VENDAS ===
Found 26 records in silver.mesclada_vendas


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,62655,GISELE BEZERRA,62655,GISELE BEZERRA GOTTARDI,160751,2025-09-02,MENSALIDADE CRIOPRESERVACAO DE OOCITOS -,1.0,131.00,Criopreservação,10150,502,29965,RPS,29848,0,CRIOPRESERVACAO,056896,,S3566.40,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,1447,20250916_153309,Mescladas2.xlsx
1,62655,GISELE BEZERRA,62655,GISELE BEZERRA GOTTARDI,160751,2025-08-04,MENSALIDADE CRIOPRESERVACAO DE OOCITOS -,1.0,131.00,Criopreservação,10150,502,24608,RPS,24508,0,CRIOPRESERVACAO,056896,,S3566.40,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,391632,20250916_153309,Mescladas2.xlsx
2,62655,GISELE BEZERRA,62655,GISELE BEZERRA GOTTARDI,160751,2025-08-01,CRIOPRESERVACAO MEDICOS INTERNOS,1.0,2594.00,Criopreservação,10150,502,24233,RPS,24133,10,EDUARDO LEME ALVES MOTTA,056896,,S3566.04,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100020,,,Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,391195,20250916_153309,Mescladas2.xlsx
3,62655,GISELE BEZERRA,62655,GISELE BEZERRA GOTTARDI,160751,2025-08-01,ANALISE DE EMBRIOES PGT-A,1.0,1100.00,PGT-A,10150,502,24233,RPS,24133,10,EDUARDO LEME ALVES MOTTA,056896,,S3563.30,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100015,,,PGT-A,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,391196,20250916_153309,Mescladas2.xlsx
4,62655,GISELE BEZERRA,62655,GISELE BEZERRA GOTTARDI,160751,2025-08-01,BIOPSIA POR EMBRIOES - MEDICOS INTERNOS,1.0,1880.00,Biópsia Embrionária,10150,502,24233,RPS,24133,10,EDUARDO LEME ALVES MOTTA,056896,,S3563.09,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100024,,,Biópsia - Embriões,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,391197,20250916_153309,Mescladas2.xlsx
5,62655,GISELE BEZERRA,62655,GISELE BEZERRA GOTTARDI,160751,2025-07-28,FIV - MEDICOS INTERNOS,1.0,25371.88,FIV,10150,502,23851,RPS,23750,10,EDUARDO LEME ALVES MOTTA,056896,,S3551,0.0,0.0,0.0,5202.12,Ibirapuera,0,0,31110100001,,,FIV,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,390638,20250916_153309,Mescladas2.xlsx
6,62655,GISELE BEZERRA,62655,GISELE BEZERRA GOTTARDI,160751,2025-07-28,ZYMOT,1.0,954.00,Zymot,10150,502,23851,RPS,23750,10,EDUARDO LEME ALVES MOTTA,056896,,S3573.04,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100017,,,Zymot,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,390639,20250916_153309,Mescladas2.xlsx
7,62655,GISELE BEZERRA,62655,GISELE BEZERRA GOTTARDI,160751,2025-07-28,EMBRYOSCOPE,1.0,2651.00,Embryoscope,10150,502,23851,RPS,23750,10,EDUARDO LEME ALVES MOTTA,056896,,S3563.17,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100027,,,Embryoscope,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,390640,20250916_153309,Mescladas2.xlsx
8,62655,GISELE BEZERRA,62655,GISELE BEZERRA GOTTARDI,160751,2025-07-24,"OVIDREL FPEN 250MCG 0,5 ML",1.0,609.00,Estimulação,10150,502,22875,RPS,22776,10,EDUARDO LEME ALVES MOTTA,056896,,000000000004644,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,389592,20250916_153309,Mescladas2.xlsx
9,62802,ANDRE LUIZ VAS,62655,GISELE BEZERRA GOTTARDI,160751,2025-07-23,"OVIDREL FPEN 250MCG 0,5 ML",2.0,712.00,Estimulação,10150,510,22780,RPS,22672,10,EDUARDO LEME ALVES MOTTA,896167,,000000000004644,0.0,0.0,0.0,506.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,389445,20250916_153309,Mescladas2.xlsx



QUERY SUMMARY FOR PRONTUARIO 160751
- view_micromanipulacao: 2 records
- view_micromanipulacao_oocitos: 15 records
- view_tratamentos: 2 records
- view_embrioes_congelados: 1 records
- silver.mesclada_vendas: 26 records

Database connections closed.

Accessing individual results:
Micromanipulacao records: 2
Micromanipulacao oocitos records: 15
Tratamentos records: 2
Embrioes congelados records: 1
Mesclada vendas records: 26


## Additional Examples


In [59]:
# Example: Query multiple prontuarios
# prontuarios = ['160751', '171805', '876950']
# all_results = {}
# 
# for pront in prontuarios:
#     print(f"\nQuerying prontuario: {pront}")
#     all_results[pront] = query_patient_data(pront)
# 
# # Compare results
# for pront, results in all_results.items():
#     print(f"Prontuario {pront}: {len(results['tratamentos'])} tratamentos")
